In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("goat_dataset.csv")

In [3]:
df.drop(["owner_name" , "adhaar_number" , "goat_id"] , axis = 1 , inplace = True)

In [4]:
df.head()

,milk_in_litres,age,height,weight,pregnancy,behavior,gender,hay_grass_intake,meat_quality_of_the_goat,milk_quality_of_the_goat
0,10,2,70,35,no,docile,female,5,4,5
1,15,3,75,40,yes,normal,female,6,3,4
2,20,4,80,45,no,aggressive,male,7,5,3
3,8,1,65,30,no,docile,male,4,2,2
4,12,5,85,50,yes,normal,female,8,4,5


In [5]:
pregnancy_dict = {
    "no" : 0,
    "yes" : 1
}
df['pregnancy'] = df['pregnancy'].map(pregnancy_dict)

In [6]:
behavior_dict = {
    "docile" : 0,
    "normal" : 1,
    "aggressive" : 2
}
df['behavior'] = df['behavior'].map(behavior_dict)

In [7]:
gender_dict = {
    "female" : 0,
    "male" : 1
}
df['gender'] = df['gender'].map(gender_dict)

In [8]:
df.head()

,milk_in_litres,age,height,weight,pregnancy,behavior,gender,hay_grass_intake,meat_quality_of_the_goat,milk_quality_of_the_goat
0,10,2,70,35,0,0,0,5,4,5
1,15,3,75,40,1,1,0,6,3,4
2,20,4,80,45,0,2,1,7,5,3
3,8,1,65,30,0,0,1,4,2,2
4,12,5,85,50,1,1,0,8,4,5


In [9]:
# import pandas as pd
# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.pipeline import Pipeline

In [10]:
# numerical_columns = ["milk_in_litres" , "age" , "height" , "weight" , "hay_grass_intake"]
# categorical_columns = ["pregnancy" , "behavior" , "gender"]

In [11]:
# numerical_pipeline = Pipeline(
#     steps = [
#         ("imputer" , SimpleImputer(strategy = "median")),
#         ("scaler" , StandardScaler())
#     ]
# )

# categorical_pipeline = Pipeline(
#     steps=[
#         ("imputer" , SimpleImputer(strategy = "most_frequent")),
#         ("one_hot_encoder" , OneHotEncoder()),
#         ("scaler" , StandardScaler(with_mean = False))
#     ]
# )

# preprocessor = ColumnTransformer(
#                 [
#                     ("numerical_pipeline",numerical_pipeline,numerical_columns),
#                     ("categorical_pipeline",categorical_pipeline,categorical_columns)
#                 ]
# )

## Saving the preprocessor pipeline for inference api's

## meat_quality_of_the_milk

In [12]:
x = df.drop(["meat_quality_of_the_goat" , "milk_quality_of_the_goat"] , axis = 1 )
y = df[["meat_quality_of_the_goat"]]

In [13]:
from sklearn.model_selection import train_test_split

# Assuming X and y are your features and labels respectively
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(f"Training set size: {x_train.shape[0]}")
print(f"Test set size: {x_test.shape[0]}")


Training set size: 72
Test set size: 18


In [14]:
type(x_train)
type(x_test)

pandas.core.frame.DataFrame

In [15]:
x_train.head()

,milk_in_litres,age,height,weight,pregnancy,behavior,gender,hay_grass_intake
49,21,5,85,45,1,0,0,8
62,19,5,85,48,1,2,1,8
73,14,3,75,38,1,0,1,6
69,12,2,70,35,0,0,0,5
76,9,1,65,30,0,0,1,4


In [16]:
# x_train = preprocessor.fit_transform(x_train)
# x_test = preprocessor.transform(x_test)

In [17]:
# from joblib import dump, load

# dump(preprocessor, 'preprocessor_.pkl')

## Model Building

### Giving every model a chance along with grid search cv implementation

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [19]:
X_train = x_train
X_test = x_test

In [20]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    print(y_train_pred)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

[[2.75122568]
 [5.08033034]
 [2.57454126]
 [2.45851761]
 [2.43002742]
 [4.0778546 ]
 [2.57454126]
 [3.85541389]
 [2.75122568]
 [3.84908892]
 [4.95323941]
 [5.08861689]
 [2.44163972]
 [4.95647816]
 [4.93658301]
 [2.44163972]
 [4.07233022]
 [2.75384027]
 [5.08033034]
 [3.85541389]
 [3.71759359]
 [4.93658301]
 [3.84908892]
 [2.56150093]
 [3.85265171]
 [4.94771504]
 [2.42726523]
 [2.56988588]
 [4.07233022]
 [2.42726523]
 [3.84908892]
 [4.93658301]
 [5.00799885]
 [4.07233022]
 [2.62862736]
 [4.07233022]
 [2.43002742]
 [3.71759359]
 [2.75122568]
 [3.82958683]
 [2.57177907]
 [2.56150093]
 [2.458616  ]
 [3.85541389]
 [4.94771504]
 [5.0617471 ]
 [2.46137819]
 [5.00799885]
 [5.00799885]
 [4.94771504]
 [2.62862736]
 [3.71759359]
 [2.46137819]
 [5.08033034]
 [2.44163972]
 [2.458616  ]
 [3.82958683]
 [3.82958683]
 [2.45851761]
 [3.58045073]
 [5.08110094]
 [3.85265171]
 [5.08110094]
 [2.458616  ]
 [2.57177907]
 [3.85541389]
 [2.44163972]
 [2.57177907]
 [2.57454126]
 [4.95323941]
 [3.9693469 ]
 [3.58

C:\Users\maste\AppData\Local\Temp\ipykernel_6984\415202671.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train) # Train model


[2.04       5.         3.         2.93533333 2.         4.
 3.         3.75615476 2.04       4.         5.         5.
 2.01       5.         4.985      2.01       4.         2.59
 5.         3.75615476 3.99       4.985      4.         3.06061905
 3.99333333 4.98       2.         2.065      4.         2.
 4.         4.985      5.         4.         2.99       4.
 2.         3.99       2.04       3.99333333 3.         3.06061905
 2.97       3.75615476 4.98       5.         2.08       5.
 5.         4.98       2.99       3.99       2.08       5.
 2.01       2.97       3.99333333 3.99333333 2.93533333 3.16
 5.         3.99333333 5.         2.97       3.         3.75615476
 2.01       3.         3.         5.         4.         3.16      ]
Random Forest Regressor
Model performance for Training set
- Root Mean Squared Error: 0.2046
- Mean Absolute Error: 0.0696
- R2 Score: 0.9646
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.3049
- Mean Absolu

C:\Users\maste\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [21]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
5,Random Forest Regressor,0.884596
6,XGBRegressor,0.883090
4,Decision Tree,0.862069
7,AdaBoost Regressor,0.797135
2,Ridge,0.603746
0,Linear Regression,0.603090
3,K-Neighbors Regressor,0.348966
1,Lasso,0.013378


## KNN Performs the Best

In [22]:
    model = KNeighborsRegressor(n_neighbors=5)
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    print(y_train_pred)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

[[3.2]
 [5. ]
 [3. ]
 [2.8]
 [2. ]
 [4. ]
 [3. ]
 [3.8]
 [3.2]
 [4. ]
 [5. ]
 [5. ]
 [2.2]
 [5. ]
 [4.6]
 [2.2]
 [4. ]
 [3.4]
 [5. ]
 [3.8]
 [3.6]
 [4.6]
 [4. ]
 [2.8]
 [3.8]
 [5. ]
 [2. ]
 [2.4]
 [4. ]
 [2. ]
 [4. ]
 [4.6]
 [3.8]
 [4. ]
 [4.2]
 [4. ]
 [2. ]
 [3.6]
 [3.2]
 [4.4]
 [3. ]
 [2.8]
 [2.6]
 [3.8]
 [5. ]
 [5. ]
 [2.4]
 [3.8]
 [3.8]
 [5. ]
 [4.2]
 [3.6]
 [2.4]
 [5. ]
 [2.2]
 [2.6]
 [4.4]
 [4.4]
 [2.8]
 [3. ]
 [4.2]
 [3.8]
 [4.2]
 [2.6]
 [3. ]
 [3.8]
 [2.2]
 [3. ]
 [3. ]
 [5. ]
 [4. ]
 [3. ]]
AdaBoost Regressor
Model performance for Training set
- Root Mean Squared Error: 0.5072
- Mean Absolute Error: 0.3139
- R2 Score: 0.7824


In [23]:
    model = LinearRegression()
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    print(y_train_pred)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

[[2.75122568]
 [5.08033034]
 [2.57454126]
 [2.45851761]
 [2.43002742]
 [4.0778546 ]
 [2.57454126]
 [3.85541389]
 [2.75122568]
 [3.84908892]
 [4.95323941]
 [5.08861689]
 [2.44163972]
 [4.95647816]
 [4.93658301]
 [2.44163972]
 [4.07233022]
 [2.75384027]
 [5.08033034]
 [3.85541389]
 [3.71759359]
 [4.93658301]
 [3.84908892]
 [2.56150093]
 [3.85265171]
 [4.94771504]
 [2.42726523]
 [2.56988588]
 [4.07233022]
 [2.42726523]
 [3.84908892]
 [4.93658301]
 [5.00799885]
 [4.07233022]
 [2.62862736]
 [4.07233022]
 [2.43002742]
 [3.71759359]
 [2.75122568]
 [3.82958683]
 [2.57177907]
 [2.56150093]
 [2.458616  ]
 [3.85541389]
 [4.94771504]
 [5.0617471 ]
 [2.46137819]
 [5.00799885]
 [5.00799885]
 [4.94771504]
 [2.62862736]
 [3.71759359]
 [2.46137819]
 [5.08033034]
 [2.44163972]
 [2.458616  ]
 [3.82958683]
 [3.82958683]
 [2.45851761]
 [3.58045073]
 [5.08110094]
 [3.85265171]
 [5.08110094]
 [2.458616  ]
 [2.57177907]
 [3.85541389]
 [2.44163972]
 [2.57177907]
 [2.57454126]
 [4.95323941]
 [3.9693469 ]
 [3.58

In [24]:
import pickle

In [25]:
with open('meat_quality_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [26]:
import uvicorn 
from fastapi import FastAPI
import numpy as np 
import pickle 
import pandas as pd
from pydantic import BaseModel
import logging
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [27]:

pregnancy_dict = {
    "no" : 0,
    "yes" : 1
}
behavior_dict = {
    "docile" : 0,
    "normal" : 1,
    "aggressive" : 2
}
gender_dict = {
    "female" : 0,
    "male" : 1
}

In [28]:
class GoatClass(BaseModel):
    MilkInLitres: int
    Age: int
    Height: int
    Weight: int
    Pregnancy: str
    Behavior: str
    Gender: str
    HayGrassIntake: int

In [29]:
data = {
  "MilkInLitres": 10,
  "Age": 2,
  "Height": 70,
  "Weight": 35,
  "Pregnancy": "no",
  "Behavior": "docile",
  "Gender": "female",
  "HayGrassIntake": 5
}

In [30]:
    # milk_in_liters = data["MilkInLitres"]
    # print("milk: " , milk_in_liters)
    # age = data["Age"]
    # print("age: " , age)
    # height = data["Height"]
    # print("height: " , height)
    # weight = data["Weight"]
    # print("weight: " , weight)
    # pregnancy = pregnancy_dict[data["Pregnancy"]]
    # print("pregnancy: " , pregnancy)
    # behavior = behavior_dict[data["Behavior"]]
    # print("behavior: " , behavior)
    # gender = gender_dict[data["Gender"]]
    # print("gender: " , gender)
    # hay_grass_intake = data["HayGrassIntake"]
    # print("hay grass index: " , hay_grass_intake)

In [31]:
with open("meat_quality_model.pkl", "rb") as f:
    model = pickle.load(f)

In [32]:
input_arr = [[milk_in_liters , age , height , weight , pregnancy , behavior , gender , hay_grass_intake]]


NameError: name 'milk_in_liters' is not defined

In [ ]:
input_arr

In [ ]:
prediction = model.predict(input_arr)
print(int(prediction[0][0]))